In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

In [22]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
def extrair_links():
    url_base = 'https://drogariaiguatemi.com.br/'
    categorias = ['farmacia', 'saude-e-bem-estar', 'beleza', 'skincare', 'mamae-e-bebe', 'autocuidado', 'homem']
    lista_de_links = []

    for categoria in categorias:
        pagina = 1  # reinicia a contagem de páginas para cada categoria

        while True:
            url = f"{url_base}{categoria}.html?p={pagina}"
            response = requests.get(url, headers=headers)

            # Verificar se a solicitação foi bem-sucedida
            if response.status_code != 200:
                print(f"Falha na solicitação para {url}. Código de status: {response.status_code}")
                break

            soup = BeautifulSoup(response.text, 'html.parser')
            a_tags = soup.select('.product-item-info a.product-item-link')

            if not a_tags:
                print(f"Nenhum produto encontrado para a categoria {categoria} na página {pagina}. Passando para a próxima categoria.")
                break

            for a_tag in a_tags:
                link = a_tag.get('href')
                lista_de_links.append(link)

            # Verificar se há um botão "Avançar" na página
            next_page_button = soup.find('li', class_='item pages-item-next')
            if not next_page_button:
                print(f"Chegou à última página para a categoria {categoria}. Indo para a próxima categoria.")
                break

            pagina += 1

    return lista_de_links

def extrair_infos(url):
    try: 
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
            
        nome_elemento = soup.find('h1', class_='page-title')
        ean_elemento = soup.find('td', {'class': 'col data', 'data-th': 'EAN'})
        marca_elemento = soup.find('td', {'class': 'col data', 'data-th': 'Marca'})
        descricao_elemento = soup.find('div', class_='product attribute description')

        nome = nome_elemento.find('span', class_='base').text.strip()
        ean = ean_elemento.text.strip()
        marca = marca_elemento.text.strip() if marca_elemento else ''
        descricao = descricao_elemento.find('div', class_='value').text if descricao_elemento else ''
            
        preco_especial = soup.find('span', class_='special-price')
        if preco_especial:
            preco_desconto = preco_especial.find('span', class_='price').text.strip()
            preco_normal_elemento = soup.find('span', class_='old-price')
            preco_normal = preco_normal_elemento.find('span', class_='price').text.strip()
        else:
            preco_normal_elemento = soup.find('span', class_='price-container price-final_price tax weee')
            preco_normal = preco_normal_elemento.find('span', class_='price').text.strip()
            preco_desconto = preco_normal

        informacoes = {
            'Produto': nome,
            'EAN': ean,
            'Marca': marca,
            'Descrição': descricao,
            'Preço sem desconto': preco_normal,
            'Preço com desconto': preco_desconto
        }
    except Exception as e:
        print(f"Erro ao extrair informações de {url}: {str(e)}")
        return None
    return informacoes

def process_url(url):
    info = extrair_infos(url)
    if info:
        x = process_url.counter
        process_url.counter += 1
        print(f"Processed {x + 1} of {len(df_links)} links: {url}")
        return info  # Return the info instead of appending to infos list

# Set an initial counter value
process_url.counter = 0

In [20]:
x = 0

In [10]:
lista_links = extrair_links()

https://drogariaiguatemi.com.br/cathy-beauty-sombra-uno-eyestalk-crytal-cream.html farmacia 1
https://drogariaiguatemi.com.br/cathy-beauty-sombra-uno-eyestalk-bluetiful.html farmacia 1
https://drogariaiguatemi.com.br/cathy-beauty-sombra-uno-eyestalk-dancing-queen.html farmacia 1
https://drogariaiguatemi.com.br/cathy-beauty-sombra-uno-eyestalk-lilac.html farmacia 1
https://drogariaiguatemi.com.br/cathy-beauty-sombra-uno-eyestalk-fabulous.html farmacia 1
https://drogariaiguatemi.com.br/cathy-beauty-sombra-uno-eyestalk-sexy-shimmer.html farmacia 1
https://drogariaiguatemi.com.br/cathy-beauty-lapis-de-olhos-cateyes-sunrise-blue.html farmacia 1
https://drogariaiguatemi.com.br/cathy-beauty-lapis-de-olhos-cateyes-gray-metalic.html farmacia 1
https://drogariaiguatemi.com.br/cathy-beauty-lapis-de-olhos-cateyes-bronze-metalic.html farmacia 1
https://drogariaiguatemi.com.br/cathy-beauty-lapis-de-olhos-cateyes-dark-brown.html farmacia 1
https://drogariaiguatemi.com.br/cathy-beauty-lapis-de-olhos-c

In [14]:
df_links = pd.DataFrame(lista_links)
df_links = df_links.rename(columns={0: 'URL'})
df_links.to_csv('df_links.csv')

In [23]:
# Visitaremos cada link e puxaremos as informações desejadas do html
infos = []

with ThreadPoolExecutor() as executor:
    results = list(executor.map(process_url, df_links['URL']))

# Filter out None values from results and append to infos
infos.extend(filter(None, results))
df_infos = pd.DataFrame(infos)
df_infos.to_csv('df_infos.csv')

Processed 1 of 8728 links: https://drogariaiguatemi.com.br/cathy-beauty-sombra-uno-eyestalk-crytal-cream.html
Processed 2 of 8728 links: https://drogariaiguatemi.com.br/cathy-beauty-sombra-uno-eyestalk-sexy-shimmer.html
Processed 3 of 8728 links: https://drogariaiguatemi.com.br/cathy-beauty-sombra-uno-eyestalk-bluetiful.html
Processed 4 of 8728 links: https://drogariaiguatemi.com.br/cathy-beauty-sombra-uno-eyestalk-lilac.html
Processed 5 of 8728 links: https://drogariaiguatemi.com.br/cathy-beauty-sombra-uno-eyestalk-fabulous.html
Processed 6 of 8728 links: https://drogariaiguatemi.com.br/cathy-beauty-lapis-de-olhos-cateyes-gray-metalic.html
Processed 7 of 8728 links: https://drogariaiguatemi.com.br/cathy-beauty-lapis-de-olhos-cateyes-sunrise-blue.html
Processed 8 of 8728 links: https://drogariaiguatemi.com.br/cathy-beauty-sombra-uno-eyestalk-dancing-queen.html
Processed 9 of 8728 links: https://drogariaiguatemi.com.br/cathy-beauty-lapis-de-olhos-cateyes-bronze-metalic.html
Processed 10

### Tratamento dos dados

In [68]:
df_infos = pd.DataFrame(infos)

In [69]:
df_infos = df_infos.drop_duplicates()
df_infos = df_infos.reset_index(drop=True)

In [70]:
# Inserindo % de desconto
df_infos['Preço sem desconto'] = df_infos['Preço sem desconto'].str.replace('R$', '').str.replace('.', '').str.replace(',', '.').astype(float)
df_infos['Preço com desconto'] = df_infos['Preço com desconto'].str.replace('R$', '').str.replace('.', '').str.replace(',', '.').astype(float)

df_infos['% de desconto'] = df_infos.apply(lambda row: round((row['Preço sem desconto'] - row['Preço com desconto']) / row['Preço sem desconto'] * 100) if pd.notna(row['Preço sem desconto']) and pd.notna(row['Preço com desconto']) and row['Preço sem desconto'] != row['Preço com desconto'] else None, axis=1)
df_infos['% de desconto'] = df_infos.apply(lambda row: str(round(row['% de desconto'])) + '%' if pd.notna(row['% de desconto']) else None, axis=1)

In [71]:
# Adicionando colunas finais
df_infos['Farmacia'] = 'Drogaria Iguatemi'
df_infos['Região'] = 'Sudeste'
df_infos['Cidade'] = 'São Paulo'

In [73]:
df_infos.to_csv('df_final.csv')